In [1]:
sc

In [2]:
data = sc.textFile('data/sales_segments.csv.gz')

In [3]:
data.take(1)

[u'rloc^departure_date_seg^booking_date^freq_flyer_flag^creator_office_id^creator_iata_number^creator_gds_code^creator_city_code^creator_lat^creator_lon^creator_country_code^creator_continent_code^creator_channel_code^booking_status_code^cabin_code^booking_class_code^pos_office_id^pos_iata_number^pos_gds_code^pos_city_code^pos_lat^pos_lon^pos_country_code^pos_continent_code^pos_channel_code^board_point_seg^board_lat^board_lon^board_country_code^board_continent_code^off_point_seg^off_lat^off_lon^off_country_code^off_continent_code^num_legs^distance_seg^route^network^airline_code^flight_number^flight_type^revenue_source^estimated_flag^usage_board_point_seg^usage_off_point_seg^usage_flight_number^usage_date^non_currency_type^bookings_seg^rpk_seg^revenue_amount_seg^currency_code^fuel_surcharge_amount_seg^fuel_surcharge_currency_code^emd_count^emd_amount^emd_currency_code^total_outliers']

In [4]:
#para eliminar la primera fila
header = data.first()

In [5]:
lines = data.filter(lambda line: line != header)

In [6]:
lines.take(1)

[u'59GS9Y^2013-12-21^2013-09-05^false^RUNHS3103^66200201^1A^RUN^-20.90083^55.53715^RE^Africa^Travel Agent^HK^Y^N^RUNHS3103^66200201^1A^RUN^-20.90083^55.53715^RE^Africa^Travel Agent^ZYR^50.8348^4.33653^BE^Europe^CDG^49.01278^2.55^FR^Europe^0^239.573363253866^CDG-ZYR^Europe^UU^5901^I^LIFT^false^ZYR^CDG^5901^2013-12-21^^1^239^33.56^EUR^7.30^EUR^0^0.00^^7.3']

In [7]:
lines.first().split('^')

[u'59GS9Y',
 u'2013-12-21',
 u'2013-09-05',
 u'false',
 u'RUNHS3103',
 u'66200201',
 u'1A',
 u'RUN',
 u'-20.90083',
 u'55.53715',
 u'RE',
 u'Africa',
 u'Travel Agent',
 u'HK',
 u'Y',
 u'N',
 u'RUNHS3103',
 u'66200201',
 u'1A',
 u'RUN',
 u'-20.90083',
 u'55.53715',
 u'RE',
 u'Africa',
 u'Travel Agent',
 u'ZYR',
 u'50.8348',
 u'4.33653',
 u'BE',
 u'Europe',
 u'CDG',
 u'49.01278',
 u'2.55',
 u'FR',
 u'Europe',
 u'0',
 u'239.573363253866',
 u'CDG-ZYR',
 u'Europe',
 u'UU',
 u'5901',
 u'I',
 u'LIFT',
 u'false',
 u'ZYR',
 u'CDG',
 u'5901',
 u'2013-12-21',
 u'',
 u'1',
 u'239',
 u'33.56',
 u'EUR',
 u'7.30',
 u'EUR',
 u'0',
 u'0.00',
 u'',
 u'7.3']

In [8]:
#variables con las que vamo a trabajar en el clustering
fields = ['bookings_seg', 'revenue_amount_seg', 'fuel_surcharge_amount_seg']

In [9]:
header.split('^').index('bookings_seg')

49

In [10]:
#iterar sobre fields para obtener los elementos correspondientes
[header.split('^').index(field) for field in fields]

[49, 51, 53]

In [16]:
def average_fare_tax(line):
    e = line.split('^')
    bookings = float(e[49])
    rev = float(e[51])
    tax = float(e[53])
    
    av_rev = rev/bookings
    av_tax = tax/bookings
    
    return (av_rev, av_tax)

In [17]:
average_fare_tax(lines.first())

(33.56, 7.3)

In [18]:
simple = lines.map(average_fare_tax)

In [19]:
simple.take(3)

[(33.56, 7.3), (61.2, 25.5), (71.86842105263158, 12.31578947368421)]

In [21]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [22]:
clusters = KMeans.train(simple, 10)

In [23]:
clusters.k

10

In [24]:
clusters.centers

[array([ 431.19140489,  184.92581272]),
 array([  8.77500428e+05,   3.03020000e+01]),
 array([  6.15206951e+05,   2.97312500e+01]),
 array([  1.08940744e+05,   2.14350000e+01]),
 array([ 109.20412744,   63.82547158]),
 array([  1.39215574e+06,   3.27400000e+01]),
 array([  4.43932834e+04,   3.30665278e+01]),
 array([  3.72063199e+05,   1.03505455e+02]),
 array([ 1569.05656347,   208.88342533]),
 array([  2.62467806e+05,   7.43850000e+01])]

In [25]:
clusters.predict((33.56, 7.3))

4

In [35]:
bycluster = simple.map(lambda point: (clusters.predict(point), point))

In [30]:
# devolvemos como clave el cluster y luego los valores av_rev y av_tax
bycluster.take(3)

[(4, (33.56, 7.3)),
 (4, (61.2, 25.5)),
 (4, (71.86842105263158, 12.31578947368421))]

In [ ]:
# versión no óptima debido al posible tamaño de cada grupo
# bycluster.groupByKey().take(3)

[(0, <pyspark.resultiterable.ResultIterable at 0x7f2f0d579cd0>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f2f0d65bc50>),
 (2, <pyspark.resultiterable.ResultIterable at 0x7f2f0d5cbcd0>)]

In [ ]:
def calc_avg_reduce()

In [33]:
means = bycluster.reduceByKey(calc_avg_reduce)

NameError: name 'calc_avg_reduce' is not defined

In [34]:
bycluster_whithones = simple.map(lambda (x,y): (clusters.predict((x,y)), (x,y,1)))

In [37]:
bycluster_whithones.cache()

PythonRDD[64] at RDD at PythonRDD.scala:43

In [38]:
bycluster_whithones.take(3)

[(4, (33.56, 7.3, 1)),
 (4, (61.2, 25.5, 1)),
 (4, (71.86842105263158, 12.31578947368421, 1))]

In [45]:
def calc_avg_reduce(acc,p):
    (rev_p, tax_p, c) = p
    (rev_ac, tax_ac, c_ac) = acc
    result = (rev_p + rev_ac, tax_p + tax_ac, c + c_ac)
    return result

In [46]:
bycluster_whithones.reduceByKey(calc_avg_reduce).take(3)

[(0, (152480491.74609438, 65394575.44836157, 353626)),
 (1, (8775004.28, 303.02, 10)),
 (2, (4921655.609999999, 237.84999999999997, 8))]

In [47]:
avs = bycluster_whithones.reduceByKey(calc_avg_reduce)

In [48]:
avs.take(3)

[(0, (152480491.74609438, 65394575.44836157, 353626)),
 (1, (8775004.28, 303.02, 10)),
 (2, (4921655.609999999, 237.84999999999997, 8))]

In [51]:
def medias(clust):
    rev_t, tax_t, count = clust
    
    
    return(rev_t / count, tax_t/count, count)

In [52]:
avs.mapValues(medias).take(3)

[(0, (431.19140489130996, 184.92581271841314, 353626)),
 (1, (877500.428, 30.302, 10)),
 (2, (615206.9512499999, 29.731249999999996, 8))]

In [53]:
averages = avs.mapValues(medias)

In [54]:
# no funciona ya que los cluster (primera columna) se ha transformado a float y no encuentra la clave para hacer el join. En una tabbla tenermos enternos y en otras float para el número del cluster
simple.join(averages).take(3)

[(0.0, (15.91, (431.19140489130996, 184.92581271841314, 353626))),
 (0.0, (15.76, (431.19140489130996, 184.92581271841314, 353626))),
 (0.0, (7.41, (431.19140489130996, 184.92581271841314, 353626)))]

In [55]:
# Tenemos los puntos con sus medias. Falta ahora las sd
bycluster_whithones.join(averages).take(3)

[(0, ((419.985, 295.34, 1), (431.19140489130996, 184.92581271841314, 353626))),
 (0, ((306.85, 159.77, 1), (431.19140489130996, 184.92581271841314, 353626))),
 (0, ((328.75, 166.25, 1), (431.19140489130996, 184.92581271841314, 353626)))]

In [60]:
def sq_diffs(p_with_cluster_mean):
    ((rev, tax, n), (av_rev, av_tax, n_cl)) = p_with_cluster_mean
    rev_sqdiff = (rev - av_rev)**2
    tax_sqdiff = (tax - av_tax)**2
    return (rev_sqdiff, tax_sqdiff, 1) #añadimos un 1 para reutilizar en el mapValues posterior  la función calc_avg_reduce definida previamente

In [61]:
points_w_means_sqdiff = bycluster_whithones.join(averages).mapValues(sq_diffs)

In [62]:
points_w_means_sqdiff.take(3)

[(0, (125.58351058797544, 12191.292753053332, 1)),
 (0, (15460.784970344674, 632.8149135238758, 1)),
 (0, (10494.241436105303, 348.78598069324204, 1))]

In [65]:
total_sqdiffs = points_w_means_sqdiff.reduceByKey(calc_avg_reduce)

In [66]:
total_sqdiffs.take(3)

[(0, (9259201218.540583, 1231441151.841926, 353626)),
 (8, (15449812306.228933, 80645461.58142972, 20172)),
 (2, (19789565668.146088, 16.646487499999974, 8))]

In [70]:
from math import sqrt
def stdevs(sqdiffs_percluster):
    sqd_rev_total, sqd_tax_total, n_cl = sqdiffs_percluster
    std_rev = sqrt(sqd_rev_total / n_cl)
    std_tax = sqrt(sqd_tax_total / n_cl)
    return(std_rev, std_tax)

In [71]:
total_sqdiffs.mapValues(stdevs).take(3)

[(0, (161.81346836246698, 59.0112395191024)),
 (8, (875.1593239336152, 63.22887959331325)),
 (2, (49736.26150524646, 1.442501624782446))]

In [72]:
#stds de cada cluster
stds_cluster = total_sqdiffs.mapValues(stdevs)

In [73]:
#
cluster_stats = averages.join(stds_cluster)

In [75]:
#vemos que hay un cluster con media y desviación 0. Tiene que ver con los grados de libertad
cluster_stats.collect()

[(0,
  ((431.19140489130996, 184.92581271841314, 353626),
   (161.81346836246698, 59.0112395191024))),
 (9, ((262467.80625, 74.385, 8), (39946.08169664359, 66.4795444102921))),
 (3,
  ((108940.74429347813, 21.435000000000002, 92),
   (12115.104061334052, 66.30331842144592))),
 (6,
  ((44393.28336111112, 33.06652777777777, 45),
   (12220.549006478257, 34.73696625363494))),
 (1, ((877500.428, 30.302, 10), (47658.199670890375, 1.7160000000000006))),
 (4,
  ((109.20412743506554, 63.825471579108125, 1085148),
   (73.76072667630477, 70.99590103982047))),
 (7,
  ((372063.1990909091, 103.50545454545455, 11),
   (39087.8175471541, 122.55136424919489))),
 (8,
  ((1569.0565634713648, 208.88342533064937, 20172),
   (875.1593239336152, 63.22887959331325))),
 (2,
  ((615206.9512499999, 29.731249999999996, 8),
   (49736.26150524646, 1.442501624782446))),
 (5, ((1392155.74, 32.74, 1), (0.0, 0.0)))]

In [77]:
cluster_stats.take(1)

[(0,
  ((431.19140489130996, 184.92581271841314, 353626),
   (161.81346836246698, 59.0112395191024)))]

In [78]:
# Asignamos a cada fila del conjunto de datos los resultados de la table del cluster anterior
bycluster.join(cluster_stats).take(3)

[(0,
  ((419.985, 295.34),
   ((431.19140489130996, 184.92581271841314, 353626),
    (161.81346836246698, 59.0112395191024)))),
 (0,
  ((306.85, 159.77),
   ((431.19140489130996, 184.92581271841314, 353626),
    (161.81346836246698, 59.0112395191024)))),
 (0,
  ((328.75, 166.25),
   ((431.19140489130996, 184.92581271841314, 353626),
    (161.81346836246698, 59.0112395191024))))]

In [85]:
full = bycluster.join(cluster_stats).cache()

In [100]:
def zscore(p_w_stats):
    
    p, stats = p_w_stats 
    # point = ((306.85, 159.77)
    #stats = ((431.19140489130996, 184.92581271841314, 353626), (161.81346836246698, 59.0112395191024))))
    rev, tax = p
    means, stdevs = stats
    rev_m, tax_m, _ =  means # Opcional: descarto n_cl, lo reemplazo por un _ ya que no lo necesito.
    rev_std, tax_std = stdevs
    
    if rev_std == 0:
        return rev, tax, 3000, True # en el caso de que una std sea 0 y para que un cluster de un sólo elemento sea considerado como outlier, le asigno un std de 3000

    zscore_rev = (rev - rev_m) / rev_std
    zscore_tax = (tax - rev_m) / tax_std
    
    zscore = sqrt(zscore_rev**2 + zscore_tax**2)
    outlier = zscore > 3
    
    
    
    return rev, tax, zscore, outlier

In [101]:
zscore(full.first()[1])

(419.985, 295.34, 2.303169095910934, False)

In [102]:
outliers = full.mapValues(zscore)

In [103]:
outliers.take(3)

[(0, (419.985, 295.34, 2.303169095910934, False)),
 (0, (306.85, 159.77, 4.663234161164518, True)),
 (0, (328.75, 166.25, 4.534092416808896, True))]

In [104]:
outliers.saveAsTextFile('outliers')